# Setup (Imports)

In [ ]:
# helpful tool for tracking the total time it takes to run the code on varied machines
import time

start_time = time.time()

In [ ]:
# import all packages used throughout the code
# This is a complete list of all packages that need to be imported for use throughout the code
import pandas as pd
from itertools import product
from datetime import datetime
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import gc
import string
import holidays
import re
from haversine import haversine, Unit
import lightgbm as lgb
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import os
import joblib

# INPUT DATA

## To be provided by Delta

In [ ]:
# Main dataset with the number of meals per meal code per cabin per flight
# Source: Delta
mealsPerFlight_historic = pd.read_csv('data/insert_path_to_mealsPerFlight_historic.csv', header=0)

# Passenger Load Data for retrieving flight times
passengerLoadData_historic = pd.read_csv("data/insert_path_to_passenger_load_data_historic.csv", header=0)

In [ ]:
# Dataset of flights you want to predict on
# Requires 1 row for each flight
# Each row requires the following features:
# 'DepartTime', 'FlightDate', 'FlightOrigin', 'FlightDestination', 'FlightNumber', 'AircraftType'
input_dataset = pd.read_csv('data/insert_path_to_input_data.csv', header=0)

### Datasets provided by MSBA team

In [ ]:
# Additional dataset that contains capacity information regarding 
# Source: Extracted data from Delta PDF (extraction by Pamela Cheng)
capacity_info = pd.read_excel('data/Fleet Key_v1.xlsx', header=0)

# Additional dataset that associate airport IATA codes with their country and coordinates
# Source: https://github.com/datasets/airport-codes/blob/master/data/airport-codes.csv
airport_codes = pd.read_csv("data/airport-codes.csv", header=0)

# Data Prep

## mealsPerFlight_historic
This is the main dataset that we are using for the model, mealsPerFlight_historic

In [ ]:
# Ensure FlightDate is formatted as datetime in the df
mealsPerFlight_historic['FlightDate'] = pd.to_datetime(mealsPerFlight_historic['FlightDate'], errors='coerce')

# Strip whitespace from string columns for merging later
mealsPerFlight_historic['FlightOrigin'] = mealsPerFlight_historic['FlightOrigin'].str.strip()
mealsPerFlight_historic['FlightDestination'] = mealsPerFlight_historic['FlightDestination'].str.strip()
mealsPerFlight_historic['EntreeType'] = mealsPerFlight_historic['EntreeType'].str.strip()

## input_dataset

In [ ]:
# Ensure FlightDate is formatted as datetime in the df
input_dataset['FlightDate'] = pd.to_datetime(input_dataset['FlightDate'], errors='coerce')

# Strip whitespace from string columns for merging later
input_dataset['FlightOrigin'] = input_dataset['FlightOrigin'].str.strip()
input_dataset['FlightDestination'] = input_dataset['FlightDestination'].str.strip()
input_dataset['EntreeType'] = input_dataset['EntreeType'].str.strip()

## passengerLoadData_historic
This is the passenger load data provided by Delta, which contains information on the takeoff time at the origin, as well as the capacity for each flight

In [ ]:
# Convert date columns to datetime format
passengerLoadData_historic['Flt_Orig_Dt'] = pd.to_datetime(passengerLoadData_historic['Flt_Orig_Dt'])
passengerLoadData_historic['Schd_Dprt_LTs'] = pd.to_datetime(passengerLoadData_historic['Schd_Dprt_LTs'])
passengerLoadData_historic['Schd_Arr_LTs'] = pd.to_datetime(passengerLoadData_historic['Schd_Arr_LTs'])
passengerLoadData_historic.drop(columns=['Index', 'Schd_Arr_LTs'], inplace=True)

# Rename columns for easier processing, merging later
flight_times = passengerLoadData_historic.rename(columns={
    'Flt_Orig_Dt': 'FlightDate',
    'Flt_Nb': 'FlightNumber',
    'Schd_Orig_Stn_Cd': 'FlightOrigin',
    'Schd_Dest_Stn_Cd': 'FlightDestination',
    'Actl_Ac_Typ_Cd': 'AircraftType',
    'Dom_Intl_Cd': 'DomOrInt',
    'Schd_Dprt_LTs': 'DepartTime',
    'Pax_Ct': 'PassengerCount',
    'Seat_Avby_Ct': 'PassengerCapacity'
})

# Create new columns based on the values in 'DomOrInt'
# Can use these as features later to count the number of domestic and international flights in an aggregation
# flight_times['IsDom'] = (flight_times['DomOrInt'] == 'D').astype(int) # is a domestic flight
# flight_times['IsInt'] = (flight_times['DomOrInt'] == 'I').astype(int) # is an international flight
# flight_times.drop(columns=['DomOrInt', 'PassengerCount'], inplace=True) # Drop the original 'DomOrInt' column

# Strip whitespace from FlightOrigin and FlightDestination -- important for merges later
flight_times['FlightOrigin'] = flight_times['FlightOrigin'].str.strip()
flight_times['FlightDestination'] = flight_times['FlightDestination'].str.strip()
flight_times['AircraftType'] = flight_times['AircraftType'].str.strip()

## capacity_info
At this time, we don't actually use the capacity_info dataset. However, depending on the format of the input data, returning to use of this DF may be important later.

In [ ]:
# ensure correct variable types in datasets
capacity_info['Subfleet'] = capacity_info['Subfleet'].astype(str)
mealsPerFlight_historic['AircraftType'] = mealsPerFlight_historic['AircraftType'].astype(str)
input_dataset['AircraftType'] = input_dataset['AircraftType'].astype(str)

In [ ]:
# Columns which have dirty data (should be int but include non-numerical values)
columns_to_process = ['C', 'F', 'W', 'Y', 'Total']

# Convert columns to string
capacity_info[columns_to_process] = capacity_info[columns_to_process].astype(str)

# Extract only the numeric part before any non-numeric characters occur (most noise is something like "151(1)", we want to remove the "(1)")
for column in columns_to_process:
    capacity_info[column] = capacity_info[column].apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else '')

# Convert columns to integers after fixing them
capacity_info[columns_to_process] = capacity_info[columns_to_process].astype(int)

## airport_codes
Here, we create a table of each airport IATA code, the country where the airport is located, and the coordinates of the airport

In [ ]:
# Select only the necessary columns from the dataset
airport_codes = airport_codes[["iata_code", 'iso_country', 'coordinates']]

# Originally, coordinates are stored as a string like "latitude, longitude"
# Instead, store them as a tuple for easier access to each attribute
airport_codes['coordinates'] = airport_codes['coordinates'].apply(lambda x: tuple(map(float, x.split(', ')))) 

# Drop rows with missing values in the "iata_code" column
# There is a lot of data we don't need here, any airport that doesn't have an IATA code is unneccesary
# All of the airports in our mealsPerFlight_historic df are identified by their IATA code
airport_codes = airport_codes.dropna(subset=["iata_code"])

## flight_distances
In order to save processing time later, we will pre-compute the distance in miles for each unique origin-destination pair by using the haversine distance based on the latitude and longitude of the origin and destination. This new table of flight distances can be merged with our mealsPerFlight_historic dataset. The flight distance can be used as a feature later.

In [ ]:
# Extract unique origins and destinations
unique_origins = mealsPerFlight_historic['FlightOrigin'].unique()
unique_destinations = mealsPerFlight_historic['FlightDestination'].unique()

# Generate all possible combinations
unique_combinations = list(itertools.product(unique_origins, unique_destinations))

# Merge with airport_codes to get coordinates for origin
flight_distances = pd.merge(unique_combinations, airport_codes, left_on='FlightOrigin', right_on='iata_code', how='left')
flight_distances.rename(columns={'coordinates': 'OriginCoordinates', 'iso_country': 'OriginCountry'}, inplace=True)

# Merge with airport_codes to get coordinates for destination
flight_distances = pd.merge(flight_distances, airport_codes, left_on='FlightDestination', right_on='iata_code', how='left')
flight_distances.rename(columns={'coordinates': 'DestinationCoordinates', 'iso_country': 'DestinationCountry'}, inplace=True)

# Use tuples of latitude and longitude for origin and destination to compute haversine distance for each unique origin-destination pair
flight_distances['FlightDistance'] = flight_distances.apply(lambda row: haversine(row['OriginCoordinates'], row['DestinationCoordinates'], unit=Unit.MILES), axis=1)

# Splitting the origin coordinates into latitude and longitude
flight_distances[['OriginLatitude', 'OriginLongitude']] = flight_distances['OriginCoordinates'].apply(lambda x: pd.Series({'OriginLatitude': x[0], 'OriginLongitude': x[1]}))

# Splitting the destination coordinates into lat and long
flight_distances[['DestinationLatitude', 'DestinationLongitude']] = flight_distances['DestinationCoordinates'].apply(lambda x: pd.Series({'DestinationLatitude': x[0], 'DestinationLongitude': x[1]}))

# Drop now unneccessary columns
flight_distances.drop(columns=['iata_code_x', 'iata_code_y', 'OriginCoordinates', 'DestinationCoordinates'], inplace=True)

# Data Pipeline Steps

### In scope FlightOrigin, MealCode
Based on the information provided by Angela on March 3rd, 2024, we must reduce the scope of the dataset. As such, we've created a list for the in scope flight origins and meal codes. Then, we use a function to remove rows that do not fall within the scope of the project.

In [ ]:
# Valid flight origins list (as provided by Delta)
valid_flight_origins = ['ANC', 'ATL', 'AUS', 'BDL', 'BNA', 'BOS', 'BWI', 'CLT', 'CMH', 'CVG', 'DCA', 'DEN', 'DFW', 'DTW', 'EWR', 'FLL', 'HNL', 'IAD', 'IAH', 'IND', 'JAX', 'JFK', 'KOA', 'LAS', 'LAX', 'LGA', 'MCI', 'MCO', 'MEM', 'MEX', 'MIA', 'MKE', 'MSP', 'MSY', 'OGG', 'ONT', 'ORD', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PVD', 'RDU', 'RSW', 'SAN', 'SAT', 'SEA', 'SFO', 'SJC', 'SLC', 'SMF', 'SNA', 'STL', 'TPA']

# Valid meal codes list (as provided by Delta)
valid_meal_codes = ['AVML', 'VLML', 'BLML', 'CHML', 'DBML', 'GFML', 'HNML', 'KSML', 'LFML', 'LSML', 'MOML', 'VGML', 'TDML', 'BBML']

# Function to filter the dataframe to only include rows where the FlightOrigin and MealCode fall into the scope of the project
def in_scope_filtering(dataframe):
    scope_df = dataframe.copy()
    scope_df = scope_df[scope_df['FlightOrigin'].isin(valid_flight_origins)] # Filter for flight origins
    scope_df = scope_df[scope_df['MealCode'].isin(valid_meal_codes)] # Filter for mealcodes

    return scope_df

### Aggregate to ignore EntreeType
In the original dataset, there is a row for each meal served for each MealCode in each CabinCode of each flight. However, we want to predict the number of customers we need to serve -- not the number of meals we need to serve to each customer (although, in the aggregate model, we come closer to that in actuality)

In [ ]:
# Function to aggregate to number of customers to serve
def aggregate_to_passenger_preference(dataframe):
    agg_df = dataframe.copy()

    # Group by the specified columns and aggregate
    agg_df = agg_df.groupby(['FlightDate', 'FlightOrigin', 'FlightDestination', 
                        'FlightNumber', 'AircraftType', 'MealCode', 
                        'CabinCode', 'CateredQuantity'], observed=True)['EntreeType'].unique().reset_index()
    
    agg_df = agg_df.drop(columns=['EntreeType']) # Drop the column we ignored in the aggregation
        
    return agg_df

### All meals on all flights
In the initial mealsPerFlight_historic dataset, in most instances when a meal was not catered for a flight, there was no row to reflect that meal. However, in actuality, if meals are offered on a flight, then all meals are available in all cabins (besides domestic flights, where only first class is served meals).
So, we must re-process the initial dataset to ensure that, for all flights, we see all meals for all cabins. In another step, we will filter out cabins and flights which won't actually receive a meal

In [ ]:
# List of unique values for MealCode and CabinCode
meal_codes = ['AVML', 'VLML', 'BLML', 'CHML', 'DBML', 'GFML', 'HNML', 'KSML', 'LFML', 'LSML', 'MOML', 'VGML', 'TDML', 'BBML']
cabin_codes = ['C', 'F', 'W', 'Y']

# Function to ensure that for all flights, there is a row for each possible combination of MealCode and CabinCode
def all_meals_all_flights(dataframe):
    all_meals_df = dataframe.copy()

    # Create a list of all unique combinations of FlightDate, FlightOrigin, FlightDestination, FlightNumber, and AircraftType
    flight_combinations = all_meals_df[['FlightDate', 'FlightOrigin', 'FlightDestination', 'FlightNumber', 'AircraftType']].drop_duplicates()

    # Create a list of all possible combinations of MealCode and CabinCode
    meal_cabin_combinations = pd.DataFrame(list(product(meal_codes, cabin_codes)), columns=['MealCode', 'CabinCode'])

    # Merge flight_combinations with meal_cabin_combinations to get all possible combinations
    all_combinations = pd.merge(flight_combinations, meal_cabin_combinations, how='cross')

    # Merge all_combinations with the original 'all_meals_df' dataframe to fill missing rows and set CateredQuantity to 0 where necessary
    all_meals_df = pd.merge(all_combinations, all_meals_df, on=['FlightDate', 'FlightOrigin', 'FlightDestination', 'FlightNumber', 'AircraftType', 'MealCode', 'CabinCode'], how='left')
    all_meals_df['CateredQuantity'] = all_meals_df['CateredQuantity'].fillna(0)

    return all_meals_df

### Country, Distance
Merge the input (mealsPerFlight_historic) dataset with the flight_distances dataset we computed at the start. This notably adds the coordinates and total distance of each flight, as well as the origin and destination country. Using the origin and destination countries, we can filter out rows that fall out of the scope of the project (rows that reflect passengers who won't receive meals)

In [ ]:
def country_association(dataframe):
    dist_df = pd.merge(dataframe, flight_distances, on=['FlightOrigin', 'FlightDestination'], how='left')

    # Drop the flights that won't be served meals at any time
    # Domestic flights (origin and destination in US) that are under 900 miles only get beverage service
    dist_df = dist_df[((dist_df['OriginCountry'] != 'US') | (dist_df['DestinationCountry'] != 'US')) | (dist_df['FlightDistance'] >= 900)]

    # We also know that domestic flights that do have meal service ONLY serve meals to first class
    dist_df = dist_df[~((dist_df['DestinationCountry'] == 'US') & (dist_df['OriginCountry'] == 'US') & (dist_df['FlightDistance'] >= 900) & (dist_df['CabinCode'] != 'C'))]

    # Becuase of the scope of the project, OriginCountry is US for all flights. 
    # Additionally, we can't use DestinationCountry because of aggregation in later steps
    dist_df.drop(columns=['OriginCountry', 'DestinationCountry'], inplace=True) 

    return dist_df

### TimeOfDay
Using the FlightDistance we computed and the DepartTime we merged in, we can ensure that the dataset reflects the "time of day" variable from the sample report structure

In [ ]:
# Apply the function to create the TimeOfDay column
def time_of_day(dataframe):
    time_df = dataframe.copy()

    # conditions = [
    #     (time_df['IsDom'] == 1) & (900 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 1499) & (500 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 945),
    #     (time_df['IsDom'] == 1) & (1500 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 2299) & (946 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 1559),
    #     (time_df['IsDom'] == 1) & (1500 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 2299) & (1600 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 2059),
    #     (time_df['IsDom'] == 1) & ((time_df['FlightDistance'] >= 2300) & ((1600 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) | (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 459))),
    #     (time_df['IsInt'] == 1) & (500 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 929),
    #     (time_df['IsInt'] == 1) & (930 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 2359)
    # ]

    conditions = [
        (time_df['DomOrInt'] == 'D') & (900 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 1499) & (500 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 945),
        (time_df['DomOrInt'] == 'D') & (1500 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 2299) & (946 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 1559),
        (time_df['DomOrInt'] == 'D') & (1500 <= time_df['FlightDistance']) & (time_df['FlightDistance'] <= 2299) & (1600 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 2059),
        (time_df['DomOrInt'] == 'D') & ((time_df['FlightDistance'] >= 2300) & ((1600 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) | (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 459))),
        (time_df['DomOrInt'] == 'I') & (500 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 929),
        (time_df['DomOrInt'] == 'I') & (930 <= time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute) & (time_df['DepartTime'].dt.hour * 100 + time_df['DepartTime'].dt.minute <= 2359)
    ]

    choices = ["Breakfast", "Lunch", "Dinner", "Dinner", "Brunch", "Lunch/Dinner"]

    time_df['TimeOfDay'] = np.select(conditions, choices, default="Unknown")

    return time_df

### Which Meals?
Creates new columns to one hot encode which meals should be served

In [ ]:
def add_meal_columns(dataframe):
    # Copy the dataframe to avoid modifying the original
    meals_df = dataframe.copy()
    
    # Add columns and set default values to 0
    meals_df['1stServiceMeal'] = 0
    meals_df['PreArrivalMeal'] = 0
    meals_df['MidflightMeal'] = 0
    
    # Compute conditions
    is_int_condition = meals_df['DomOrInt'] == 'I'
    depart_time_condition = meals_df['DepartTime'].dt.hour.between(5, 23)
    flight_distance_condition = meals_df['FlightDistance'] >= 6000
    
    # Update columns based on conditions
    meals_df.loc[is_int_condition & depart_time_condition, '1stServiceMeal'] = 1
    meals_df.loc[is_int_condition & depart_time_condition, 'PreArrivalMeal'] = 1
    meals_df.loc[is_int_condition & depart_time_condition & flight_distance_condition, 'MidflightMeal'] = 1
    
    return meals_df

### Create date related columns
Extract useful features from the FlightDate column

In [ ]:
def date_columns(dataframe):
    dates_df = dataframe.copy()

    dates_df['Year'] = dates_df['FlightDate'].dt.year
    dates_df['Month'] = dates_df['FlightDate'].dt.month
    dates_df['Day'] = dates_df['FlightDate'].dt.day
    # dates_df['DayOfYear'] = dates_df['FlightDate'].dt.dayofyear
    dates_df['WeekOfYear'] = dates_df['FlightDate'].dt.isocalendar().week
    dates_df['DayOfTheWeek'] = dates_df['FlightDate'].dt.day_name()

    return dates_df

### Dist from Holidays
Using the 'holidays' package in Python, we can find the number of days away from relevant holidays for each flight.
Do some precomputing and merging based on the unique FlightDate values in the dataset.
For the most part, the holidays seemed to correlate with each other. So, we kept the two most notable holidays which had high feature importance and which also are based on different calendars than the US calendar (and thus store different information than what we can extract from FlightDate)

In [ ]:
# List of holiday names
holiday_names = [
    'Chinese New Year (Spring Festival)',
    'Pesach'
]

# Function to add features to dataset which indicate how far each flight is from the occurence of a certain holiday within that calendar year
# So, Jan 1st would be -360 days from Christmas, because it's based on the calendar year
def add_holiday_distance(dataframe):
    unique_years = pd.to_datetime(dataframe['FlightDate']).dt.year.unique().tolist()
    unique_years = sorted(unique_years)

    # Create DataFrame
    holiday_df = pd.DataFrame(index=holiday_names, columns=unique_years)

    us_holidays = holidays.US()
    il_holidays = holidays.IL(language='en_US')
    cn_holidays = holidays.CN(language='en_US')

    for year in holiday_df.columns:
        # Generate date range for the current year
        start_date = pd.Timestamp(year, 1, 1)
        end_date = pd.Timestamp(year, 12, 31)
        date_range = pd.date_range(start=start_date, end=end_date)

        # Iterate over each day in the date range
        for date in date_range:
            il_holiday_name = il_holidays.get(date)
            if il_holiday_name in holiday_df.index:
                holiday_df.at[il_holiday_name, year] = date

            cn_holiday_name = cn_holidays.get(date)
            if cn_holiday_name in holiday_df.index:
                holiday_df.at[cn_holiday_name, year] = date

    unique_dates = dataframe['FlightDate'].unique().tolist()

    # Create an empty DataFrame with unique_dates as index
    reference_df = pd.DataFrame(index=unique_dates)

    # iterate over each date in the dataset
    for date in reference_df.index:
        year = date.year
        # Iterate over each holiday
        for holiday in holiday_df.index:
            holiday_date = holiday_df.at[holiday, year]
            # Check if holiday_date is not NaN (ie, a valid date)
            if not pd.isnull(holiday_date):
                holiday_cleaned = holiday.strip().translate(str.maketrans('', '', string.punctuation)).replace(" ", "")
                reference_df.at[date, "DistFrom" + holiday_cleaned] = (date - holiday_date).days

    reference_df.index = pd.to_datetime(reference_df.index)

    # Merge dataframe and reference_df on the FlightDate column and the index
    merged_df = pd.merge(dataframe, reference_df, left_on='FlightDate', right_index=True, how='left')
    
    return merged_df

### Ordinal Encoding
CabinCode and DayOfTheWeek have natural order to them, so it's better to perform ordinal encoding rather than label or one-hot encoding

In [ ]:
# Store mappings
cabin_mapping = {'C': 1, 'F': 2, 'W': 3, 'Y': 4}
days_mapping = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}

# Function to perform ordinal encoding on applicable features
def ordinal_encoding(dataframe):
    ord_df = dataframe.copy()

    ord_df['CabinCode'] = ord_df['CabinCode'].map(cabin_mapping)
    ord_df['DayOfTheWeek'] = ord_df['DayOfTheWeek'].map(days_mapping)

    return ord_df

### Label Encoding
For features with no natural order, we label encode them

In [ ]:
label_encoders = {}

# Simple function to label encode all categorical variables
def label_encoding(dataframe):
    lab_df = dataframe.copy()

    for column in lab_df.columns:
        if not lab_df[column].dtype.kind in ['i', 'f', 'M']:  # Check if column is not numeric
            le = LabelEncoder()
            lab_df[column] = le.fit_transform(lab_df[column])
            label_encoders[column] = le
    
    return lab_df

In [ ]:
def label_encoding_input(dataframe):
    lab_df = dataframe.copy()

    for column, le in label_encoders.items():
            if column in lab_df.columns:
                lab_df[column] = le.transform(lab_df[column])
    
    return lab_df

### People Capacity
Two functions in here; which to use depends on the features available in the input data. Currently, for the sake of our code and experiments, we use the simpler function which does a merge to bring the time and capacity data into our primary dataset

NOTE That some flights do not seem to be able to retrieve their flight capacity from either dataset. Thus, the fleet key dataset we provided may need some manual adjustmenets to best reflect the entire Delta fleet and the capacity of each cabin.

In [ ]:
def people_capacity(dataframe):
    ppl_df = pd.merge(dataframe, flight_times, on=['FlightDate', 'FlightNumber', 'FlightOrigin', 'FlightDestination', 'AircraftType'], how='left')

    # Drop rows where we are missing data
    # Small subset of rows thankfully
    ppl_df.dropna(inplace=True)

    return ppl_df

In [ ]:
# Define a function to select capacity based on CabinCode
def get_capacity(row):
    cabin_code = row['CabinCode']
    if cabin_code == 'C':
        return row['C']
    elif cabin_code == 'F':
        return row['F']
    elif cabin_code == 'W':
        return row['W']
    elif cabin_code == 'Y':
        return row['Y']
    else:
        return 0

# Function to use a supplemental df (capacity_info) to find the capacity of each cabin of each flight based on SubFleet
# requires you to load capacity_info, which has data on the capacity of each cabin of each subfleet of plane
def people_capacity_input(dataframe):
    ppl_df = pd.merge(dataframe, capacity_info, left_on='AircraftType', right_on='Subfleet', how='left')

    # Apply the function to create the new PeopleCapacity column
    ppl_df['PeopleCapacity'] = ppl_df.apply(get_capacity, axis=1)

    ppl_df.drop(['Aircraft Model', 'Subfleet', 'Configuration', 'Count', 'C', 'F', 'W', 'Y', 'Total'], axis=1, inplace=True)

    # ppl_df = ppl_df[ppl_df['PeopleCapacity'] != 0].dropna(subset=['PeopleCapacity'])

    return ppl_df

### Define Categorical Columns (For XGBoost Model)
Function to automatically assign columns as "category" type. Useful when running an XGBoost model which can use categorical features. XGBoost performance is improved when using the experimental feature to use categorical columns without needing encoding. It also makes error analysis simpler.

In [ ]:
# You must run this before running create_dummies()
def def_cat_cols(dataframe):
    cat_df = dataframe.copy()

    # Identify categorical columns
    categorical_columns = cat_df.select_dtypes(exclude=['number']).columns.tolist()

    # Label encode categorical columns
    for col in categorical_columns:
        cat_df[col] = cat_df[col].astype('category')

    return cat_df

### Dummy Variables
If dummy variables are preferred over label encoding, this function can be used.

In [ ]:
# Function to create dummy variables for all categorical columns
# REQUIRES you to run def_cat_cols() first in order to ensure that categorical columns use the correct datatype
def create_dummies(dataframe):
    dummy_df = dataframe.copy()    
    categorical_columns = dummy_df.select_dtypes(include=['category']).columns.tolist()
    dummy_df = pd.get_dummies(dummy_df, columns=categorical_columns)    
    return dummy_df

### Scale
Some of our model experimentation required scaling, this function is to make the scaling process simpler to implement.

In [ ]:
# Function that will scale your already created train, test, and validation sets
def scale_data(train, test, val):
    scaler = StandardScaler()

    # Fit the scaler on the training data and transform it
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    val_scaled = scaler.transform(val)

    return train_scaled, test_scaled, val_scaled

### Agg to origin-day
In our most recent iterations of the modeling code, we aggregate to the MealCode-CabinCode-Flight-TimeOfDay-Day level. This function simply runs the aggregation

In [ ]:
def origin_day_agg(dataframe):
    # Grouping by the desired columns and aggregating
    agg_df = dataframe.groupby(['FlightDate', 'FlightOrigin', 'CabinCode', 'MealCode', 'DomOrInt', 'OriginLatitude', 'OriginLongitude', 'Year', 'Month', 'Day', 'WeekOfYear', 'DayOfTheWeek', 'DistFromChineseNewYearSpringFestival', 'DistFromPesach', 'TimeOfDay', '1stServiceMeal', 'PreArrivalMeal', 'MidflightMeal'], observed=True)\
        .agg(
        NumFlights=('FlightNumber', 'nunique'),  
        TotalCateredQuantity=('CateredQuantity', 'sum'),  
        TotalDistance=('FlightDistance', 'sum'),
        TotalCapacity=('PassengerCapacity', 'sum'),
    ).reset_index()  # Resetting index to flatten the DataFrame
        
    return agg_df

In [ ]:
def origin_day_agg_input(dataframe):
    # Grouping by the desired columns and aggregating
    agg_df = dataframe.groupby(['FlightDate', 'FlightOrigin', 'CabinCode', 'MealCode', 'DomOrInt', 'OriginLatitude', 'OriginLongitude', 'Year', 'Month', 'Day', 'WeekOfYear', 'DayOfTheWeek', 'DistFromChineseNewYearSpringFestival', 'DistFromPesach', 'TimeOfDay', '1stServiceMeal', 'PreArrivalMeal', 'MidflightMeal'], observed=True)\
        .agg(
        NumFlights=('FlightNumber', 'nunique'),  
        TotalDistance=('FlightDistance', 'sum'),
        TotalCapacity=('PassengerCapacity', 'sum'),
    ).reset_index()  # Resetting index to flatten the DataFrame
        
    return agg_df

### Decoding
For error analysis, this simply decodes the various features of our dataframes based on our encoding functions used to create the training data

In [ ]:
def decode_dataframe(dataframe):
    decoded_dataframe = dataframe.copy()

    for column, le in label_encoders.items():
            if column in decoded_dataframe.columns:
                decoded_dataframe[column] = le.inverse_transform(decoded_dataframe[column])
    
    decoded_dataframe['CabinCode'] = decoded_dataframe['CabinCode'].map({v: k for k, v in cabin_mapping.items()})
    decoded_dataframe['DayOfTheWeek'] = decoded_dataframe['DayOfTheWeek'].map({v: k for k, v in days_mapping.items()})
    
    return decoded_dataframe

### isHub?
It was noted by Venkatesh at the GBAC that there might be a difference in ordering trends at Delta hub airports. Based on information from the Delta website, we have identified the hub airports. However, we have found the 'isHub' feature to be of low feature importance.

In [ ]:
def isHub(dataframe):
    hub_df = dataframe.copy()

    # List of IATA codes
    delta_hubs = ['AMS', 'ATL', 'BOG', 'BOS', 'DTW', 'LIM', 'LHR', 'LAX', 'MEX', 'MSP', 'JFK', 'LGA', 
                'CDG', 'SLC', 'SCL', 'GRU', 'SEA', 'ICN', 'NRT', 'HND']

    # Create a new column "IsHub" initialized with zeros
    hub_df['IsHub'] = 0

    # Set IsHub to 1 for rows where FlightOrigin is in the list of IATA codes
    hub_df.loc[hub_df['FlightOrigin'].isin(delta_hubs), 'IsHub'] = 1
    
    return hub_df

# Historic Data Pipeline
Here is where we run the current data processing pipeline as we have found to be most effective. 

In [ ]:
processed_historic_data = (mealsPerFlight_historic.pipe(in_scope_filtering)
                               .pipe(aggregate_to_passenger_preference) # In original dataset, customers are represented for every meal they are served. This aggregates down to the number of customer preferences, not the number of meals served.
                               .pipe(all_meals_all_flights) # Ensure each meal is represented in each cabin of each flight. 
                               .pipe(people_capacity) # Deletes any rows where we can't find a capacity (NAN), or when capacity is 0
                               .pipe(country_association) # Get country association, flight distances, and filter data based on flights which actually receive meals
                               .pipe(time_of_day) # Get the time of day and other time-based features which dictates which meals are served
                               .pipe(add_meal_columns) # For int'l flights, keep tracks of which meals will be received
                               .pipe(date_columns) # Create features from FlightDate
                               .pipe(add_holiday_distance) # Create distance from Passover and from Chinese New Year as features
                               .pipe(origin_day_agg) # Aggregate to MealCode-CabinCode-Flight-TimeOfDay-Date level
                               .pipe(isHub) # Identify if the airport is a Delta hub
                               .pipe(ordinal_encoding) # perform ordinal encoding for CabinCode, DayOfTheWeek
                               .pipe(label_encoding) # perform label encoding for non-ordinal and non-numerical columns
                               .sort_values(by=['FlightDate', 'FlightOrigin', 'CabinCode', 'MealCode']) # sort for a date-based train-test split
                               )

X_train = processed_historic_data.drop(columns=['TotalCateredQuantity', 'FlightDate'])
y_train = processed_historic_data['TotalCateredQuantity']

gc.collect()

# Input Data Pipeline

In [ ]:
processed_input_data = (input_dataset
                               .pipe(all_meals_all_flights) # Ensure each meal is represented in each cabin of each flight. 
                               .pipe(people_capacity_input) # Deletes any rows where we can't find a capacity (NAN), or when capacity is 0
                               .pipe(country_association) # Get country association, flight distances, and filter data based on flights which actually receive meals
                               .pipe(time_of_day) # Get the time of day and other time-based features which dictates which meals are served
                               .pipe(add_meal_columns) # For int'l flights, keep tracks of which meals will be received
                               .pipe(date_columns) # Create features from FlightDate
                               .pipe(add_holiday_distance) # Create distance from Passover and from Chinese New Year as features
                               .pipe(origin_day_agg_input) # Aggregate to MealCode-CabinCode-Flight-TimeOfDay-Date level
                               .pipe(isHub) # Identify if the airport is a Delta hub
                               .pipe(ordinal_encoding) # perform ordinal encoding for CabinCode, DayOfTheWeek
                               .pipe(label_encoding_input) # perform label encoding for non-ordinal and non-numerical columns
                               .sort_values(by=['FlightDate', 'FlightOrigin', 'CabinCode', 'MealCode']) # sort for a date-based train-test split
                               .drop(columns=['FlightDate']) # Drop FlightDate
                               )

# Create a decoded X dataset for easier output later
processed_input_decoded = decode_dataframe(processed_input_data)

gc.collect()

# Adding Time series feature

In [ ]:
# Initialize SARIMA model
model = SARIMAX(y_train, order=(1, 1, 0), seasonal_order=(1, 0, 0, 12)) 
# Fit the model
model_fit = model.fit(disp=False)

# Make predictions
# Predict for X_train
train_predictions = model_fit.predict(start=0, end=len(X_train)-1)
# Predict for processed_input_data
input_predictions = model_fit.predict(start=len(X_train), end=len(X_train)+len(processed_input_data)-1)

# Add train_predictions as a new column to X_train
X_train['ARIMAForecast'] = train_predictions

# Add test_predictions as a new column to processed_input_data
processed_input_data['ARIMAForecast'] = input_predictions

# LGBM
When you use the LGBM model for prediction, it provides balanced results. However, while in real life we're limited to ordering an integer number of meals, the model is a regressor and outputs a float. So, we must round. This rounding unfortunately does result in a greater frequency of stockouts compared to overstocking. 

Fortunately, we have found that adding a small value to all predictions -- some number between 0.15-0.25 seems to produce the best results -- can balance the overall error of the model with reducing stockouts. You can adjust this parameter in 'epsilon' to achieve the best results.

In [ ]:
# You can adjust the value of epsilon to fine tune the over vs. underprediction of the model
epsilon = 0

In [ ]:
lgb_model = lgb.LGBMRegressor(verbosity=-1)
lgb_model.fit(X_train, y_train)

# Save the model to disk
joblib.dump(lgb_model, 'lgb_model.pkl')

In [ ]:
# Load the model from disk if desired
# lgb_model = joblib.load('lgb_model.pkl')

# Printing Output

In [ ]:
y_pred = np.round(lgb_model.predict(processed_input_data) + epsilon)

# Create a DataFrame to store the results more neatly
results_df = pd.DataFrame({
    'Date': pd.to_datetime(processed_input_decoded[['Year', 'Month', 'Day']]),
    'ISO Dep Week': processed_input_decoded['WeekOfYear'],
    'Origin': processed_input_decoded['FlightOrigin'],
    'Cabin': processed_input_decoded['CabinCode'],
    'SPML Code': processed_input_decoded['MealCode'],
    'Domestic/International/TCON': processed_input_decoded['DomOrInt'],
    'Breakfast/Lunch/Dinner/PreArrival, etc.': processed_input_decoded['TimeOfDay'],
    '1stServiceMeal': processed_input_decoded['1stServiceMeal'],
    'PreArrivalMeal': processed_input_decoded['PreArrivalMeal'],
    'MidflightMeal': processed_input_decoded['MidflightMeal'],  
    'Predicted Demand (Meals)': y_pred
    })

In [ ]:
# Create an empty list to store the duplicated rows
duplicated_rows = []

# Iterate through each row of the DataFrame
for index, row in results_df.iterrows():
    if (row['1stServiceMeal'] == 1) or (row['PreArrivalMeal'] == 1) or (row['MidflightMeal'] == 1):
        # Check the values of the three columns
        if row['1stServiceMeal'] == 1:
            # Duplicate the row and add the meal type
            new_row = row.copy()
            new_row['MealTime'] = '1st Service Meal'
            duplicated_rows.append(new_row)
        if row['PreArrivalMeal'] == 1:
            # Duplicate the row and add the meal type
            new_row = row.copy()
            new_row['MealTime'] = 'Pre-Arrival Meal'
            duplicated_rows.append(new_row)
        if row['MidflightMeal'] == 1:
            # Duplicate the row and add the meal type
            new_row = row.copy()
            new_row['MealTime'] = 'Midflight Meal'
            duplicated_rows.append(new_row)
    elif (row['1stServiceMeal'] == 0) and (row['PreArrivalMeal'] == 0) or (row['MidflightMeal'] == 0):
        duplicated_rows.append(row)

In [ ]:
# Concatenate the original DataFrame with the duplicated rows
transformed_df = pd.DataFrame(duplicated_rows)
transformed_df['MealTime'] = transformed_df['MealTime'].fillna('').copy()
transformed_df = transformed_df.drop(columns=['1stServiceMeal', 'PreArrivalMeal', 'MidflightMeal']).copy()

In [ ]:
transformed_df['Date'] = transformed_df['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
today_date = datetime.now().strftime('%m-%d-%Y')

# Create the 'Output' directory if it doesn't exist
output_dir = f'Output (Generated {today_date})'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Group the DataFrame by 'Date' and 'Origin'
grouped = transformed_df.groupby(['Date', 'Origin'])

# Iterate over each group
for (date, origin), group_df in grouped:
    # Create a folder for each date if it doesn't exist
    date_folder = os.path.join(output_dir, date)
    if not os.path.exists(date_folder):
        os.makedirs(date_folder)

    # Sort the group DataFrame by the specified columns
    sorted_group = group_df.sort_values(by=['Breakfast/Lunch/Dinner/PreArrival, etc.', 'Cabin', 'SPML Code'])

    # Create a filename for the Excel file
    filename = f"{origin}_{date}.xlsx"
    filepath = os.path.join(date_folder, filename)

    # Write the sorted DataFrame to an Excel file
    with pd.ExcelWriter(filepath, engine='xlsxwriter') as writer:
        sorted_group.to_excel(writer, index=False, sheet_name='Sheet1')

# End Timestamp

In [ ]:
end_time = time.time()

elapsed_time_seconds = end_time - start_time
elapsed_minutes = int(elapsed_time_seconds // 60)
elapsed_seconds = int(elapsed_time_seconds % 60)

print("Elapsed time:", elapsed_minutes, "minutes and", elapsed_seconds, "seconds")